## Читаем данные с двух листов и кладём в датафрейм

In [1]:
import pandas as pd
import nltk

from cleantext import clean
from tqdm import tqdm, tqdm_pandas
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud, STOPWORDS
from matplotlib import pyplot as plt

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


In [175]:
df_excel =  pd.read_excel('st-data-full.xlsx', sheet_name=['stocktwits_1', 'stocktwits_2'])
df_excel

{'stocktwits_1':                                                      text  label
 0       if you were curious, price chose the lowest ch...      1
 1                  true, not even 10k followers here yet.      1
 2       dogecoin co-founder billy markus hits back at ...      1
 3        i’m curious, do any bulls have a price where ...      1
 4                  friday everybody buy 10 more on friday      2
 ...                                                   ...    ...
 599995  so did i 850% with ! get in below a dollar. st...      2
 599996  notice it’s the same few brain dead people bas...      2
 599997  attention!!! anyone under the age of 18…. stea...      2
 599998   dickhead bears fucked. keep your big mouths s...      2
 599999         hit that shit and make it turn like turnip      2
 
 [600000 rows x 2 columns],
 'stocktwits_2':                                                      text  label
 0        trading at very low volumes, far less liquid ...      0
 1            

In [176]:
df = pd.concat([df_excel['stocktwits_1'], df_excel['stocktwits_2']], axis=0)

## Удаляем дубликаты и пустые твиты

In [177]:
df = df.dropna().drop_duplicates()

In [178]:
df = df.reset_index(drop=True)

In [181]:
df[df['label'] == 0].sample(10)

,text,label
155552,"this has no use case, it is a speculative ass...",0
754152,just keep on shorting. 😊,0
434837,think smart and short,0
551149,straight back down dummies 🤣🧨🧨🧨🧨🧨🧨🧨🩸,0
109343,6 month low! how is it a bull market still!?,0
791063,lmao i love this i love shattering hopes and ...,0
799341,"they said 100k when in reality, it should be ...",0
92481,holy shit - rewards program shuttered???,0
208129,who thinks bloodbath tomorrow? not bearish on ...,0
533903,volatility kills crypto now.,0


In [182]:
df[df['label'] == 0].sample(10)

,text,label
989592,"when you hear, “support is bla bla”.",0
146393,fox business: nft of jack dorsey's first tweet...,0
863188,"you know, inflation is in your head",0
1101643,pumping on 0 volume,0
468004,i don’t know… better be ’s just too boring now,0
845024,last time bitcoin was in this position it went...,0
62565,wow. not even the floor either.,0
138032,i call it the toe = taper + omicron +evergran...,0
35951,taking munger’s and buffets advice,0
200646,mooboys now is the time to load up on this pu...,0


In [183]:
df[df['label'] == 1].sample(10)

,text,label
741172,china gave up z………...,1
1263089,crypto exchanges are just the fee vultures of...,1
224748,3.6m americans to use crypto to make a purchas...,1
917185,"lol, the wallets will not have any impact on t...",1
367718,when do we fall off the cliff? 😝,1
556509,berry silbert shorting it down to 1 billion. t...,1
1184754,f yea war’s on and btc pumping,1
800946,so it was a bull trap,1
126602,that price action was a transaction. as soon ...,1
665267,what’s happening here?? why it dropped?? any n...,1


In [184]:
df[df['label'] == 2].sample(10)

,text,label
177336,"bears need to learn that in the long term, yo...",2
597482,"we hit the magic number, 🚀",2
389425,what’s the news with this one anyone knows,2
231639,when people tell me bitcoin isn’t going to,2
1025139,what goes down musk go up!!,2
234144,it’s insane that covid fears are still a thing.,2
626984,the entire reason the price is being like a li...,2
368314,gbtc is a sure win for the next 4 months,2
667340,you guys need to chil the fuck out we need it ...,2
1043276,come to in the mean time 18 mill mc low supply...,2


In [185]:
df.loc[951956].text

' this ain’t even the bounce the real bounce is at all time high'

In [186]:
df.reset_index(drop=True)

,text,label
0,"if you were curious, price chose the lowest ch...",1
1,"true, not even 10k followers here yet.",1
2,dogecoin co-founder billy markus hits back at ...,1
3,"i’m curious, do any bulls have a price where ...",1
4,friday everybody buy 10 more on friday,2
...,...,...
1326854,i tried well now the haters are -45% or liquid...,0
1326855,i'd be pretty happy if bitcoin ended the year...,2
1326856,will jump to 88 000 in no time 😬✈️✈️✈️,2
1326857,"set it and forget it, i’ll see you guys at 😉😉",2


In [188]:
%%time
tqdm.pandas()
#удалим эмодзи
df['cleaned_text'] = df.text.progress_apply(clean, no_emoji=True,
                                                  no_punct=True, 
                                                  no_currency_symbols=True,
                                                  no_digits=True,
                                                  no_numbers=True)
df['cleaned_text']

100%|██████████████████████████████| 1326859/1326859 [02:02<00:00, 10875.40it/s]


CPU times: user 2min, sys: 1.39 s, total: 2min 1s
Wall time: 2min 2s


0          if you were curious price chose the lowest cha...
1                       true not even 00k followers here yet
2          dogecoin cofounder billy markus hits back at m...
3          im curious do any bulls have a price where the...
4               friday everybody buy <number> more on friday
                                 ...                        
1326854    i tried well now the haters are <number> or li...
1326855    id be pretty happy if bitcoin ended the year a...
1326856                     will jump to <number> in no time
1326857             set it and forget it ill see you guys at
1326858                            man did this came to life
Name: cleaned_text, Length: 1326859, dtype: object

In [189]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1326859 entries, 0 to 1326858
Data columns (total 3 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   text          1326859 non-null  object
 1   label         1326859 non-null  int64 
 2   cleaned_text  1326859 non-null  object
dtypes: int64(1), object(2)
memory usage: 30.4+ MB


In [190]:
df['cleaned_text'] = df['cleaned_text'].str.replace('\d+\w', '')

/var/folders/tm/0mr7bwxj05qfsq9gn9fl6bfc0000gn/T/ipykernel_90091/1261350415.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['cleaned_text'] = df['cleaned_text'].str.replace('\d+\w', '')


In [192]:
df['tokenized_text'] = df.progress_apply(lambda row: nltk.word_tokenize(row['cleaned_text']), axis=1)

100%|███████████████████████████████| 1326859/1326859 [03:23<00:00, 6518.30it/s]


In [193]:
df['tokenized_text']

0          [if, you, were, curious, price, chose, the, lo...
1                    [true, not, even, followers, here, yet]
2          [dogecoin, cofounder, billy, markus, hits, bac...
3          [im, curious, do, any, bulls, have, a, price, ...
4          [friday, everybody, buy, <, number, >, more, o...
                                 ...                        
1326854    [i, tried, well, now, the, haters, are, <, num...
1326855    [id, be, pretty, happy, if, bitcoin, ended, th...
1326856         [will, jump, to, <, number, >, in, no, time]
1326857    [set, it, and, forget, it, ill, see, you, guys...
1326858                     [man, did, this, came, to, life]
Name: tokenized_text, Length: 1326859, dtype: object

In [194]:
stopwords = nltk.corpus.stopwords.words('english')

In [196]:
stopwords.append('>')
stopwords.append('<')
stopwords.append('number')

In [197]:
def remove_stopwords(text):
    output= [i for i in text if i not in stopwords]
    return output

In [198]:
df['no_stopwords']= df['tokenized_text'].progress_apply(lambda x:remove_stopwords(x))

100%|██████████████████████████████| 1326859/1326859 [00:39<00:00, 33760.25it/s]


In [199]:
df['no_stopwords']

0          [curious, price, chose, lowest, chamber, good,...
1                               [true, even, followers, yet]
2          [dogecoin, cofounder, billy, markus, hits, bac...
3          [im, curious, bulls, price, pull, either, posi...
4                           [friday, everybody, buy, friday]
                                 ...                        
1326854                    [tried, well, haters, liquidated]
1326855    [id, pretty, happy, bitcoin, ended, year, less...
1326856                                         [jump, time]
1326857                        [set, forget, ill, see, guys]
1326858                                    [man, came, life]
Name: no_stopwords, Length: 1326859, dtype: object

In [200]:
#defining the object for Lemmatization
wordnet_lemmatizer = WordNetLemmatizer()

In [201]:
#defining the function for lemmatization
def lemmatizer(text):
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemm_text
df['text_lemmatized']=df['no_stopwords'].progress_apply(lambda x:lemmatizer(x))

100%|██████████████████████████████| 1326859/1326859 [01:26<00:00, 15319.35it/s]


In [202]:
df['text_lemmatized']

0          [curious, price, chose, lowest, chamber, good,...
1                                [true, even, follower, yet]
2          [dogecoin, cofounder, billy, markus, hit, back...
3          [im, curious, bull, price, pull, either, posit...
4                           [friday, everybody, buy, friday]
                                 ...                        
1326854                     [tried, well, hater, liquidated]
1326855    [id, pretty, happy, bitcoin, ended, year, le, ...
1326856                                         [jump, time]
1326857                         [set, forget, ill, see, guy]
1326858                                    [man, came, life]
Name: text_lemmatized, Length: 1326859, dtype: object

In [203]:
df.loc[:,['text','label', 'text_lemmatized']].to_csv('preprocessed_text.csv')